In [76]:
import pandas as pd

In [77]:
base_df = pd.read_parquet('../bin/scraped_latin_texts_words.parquet')

In [78]:
word_df = base_df.copy()
word_df = pd.DataFrame(word_df.groupby('word')['count'].sum()).reset_index()
word_df['frequency'] = word_df['count'] / sum(word_df['count'])
word_df['frequency_rank'] = word_df['count'].rank(method='max', ascending=False).astype(int)

merge_df = base_df.copy()
merge_df = pd.DataFrame(merge_df['word'].value_counts())
merge_df['cross_text_count'] = merge_df['word']
merge_df['word'] = merge_df.index
merge_df = merge_df.reset_index().drop(columns='index')

word_df = word_df.merge(merge_df, on='word', how='outer')

word_df['cross_text_frequency'] = word_df['cross_text_count'] / len(base_df.groupby(['author', 'title'])['title'])


merge_df = base_df.copy()
merge_df = pd.DataFrame(merge_df.groupby(['author', 'word'])['count'].sum()).reset_index()
merge_df = pd.DataFrame(merge_df['word'].value_counts())
merge_df['cross_author_count'] = merge_df['word']
merge_df['word'] = merge_df.index
merge_df = merge_df.reset_index().drop(columns='index')

word_df = word_df.merge(merge_df, on='word', how='outer')

word_df['cross_author_frequency'] = word_df['cross_author_count'] / len(base_df['author'].unique())

In [79]:
author_df = base_df.copy()
author_df = author_df.groupby(['author']).agg({'count': 'sum', 'word': 'nunique'}).rename(columns={'count': 'word_count', 'word': 'author_hapax_count'}).reset_index()

base_df_copy = base_df.copy()
author_word_pairs = base_df_copy[['author', 'word']].drop_duplicates()
word_author_counts = author_word_pairs.groupby('word')['author'].nunique()
unique_words = word_author_counts[word_author_counts == 1].index
unique_author_words = author_word_pairs[author_word_pairs['word'].isin(unique_words)]
merge_df = pd.DataFrame({'author': base_df['author'].unique()}).merge(unique_author_words.groupby('author').size().reset_index(name='cross_author_unique_word_count'), on='author', how='left')
merge_df['cross_author_unique_word_count'] = merge_df['cross_author_unique_word_count'].fillna(0).astype(int)

author_df = author_df.merge(merge_df, on='author', how='left')

In [80]:
text_df = base_df.copy()
text_df = text_df.groupby(['author', 'source', 'title']).agg({'count': 'sum', 'word': 'nunique'}).rename(columns={'count': 'word_count', 'word': 'text_hapax_count'}).reset_index().drop(columns=('source'))

base_df_copy = base_df.copy()
text_word_pairs = base_df_copy[['author', 'title', 'word']].drop_duplicates()
text_word_counts = (text_word_pairs.groupby('word').apply(lambda x: x[['author', 'title']].drop_duplicates().shape[0]).reset_index(name='num_texts_using_word'))
unique_words = text_word_counts[text_word_counts['num_texts_using_word'] == 1]['word']
unique_text_words = text_word_pairs[text_word_pairs['word'].isin(unique_words)]
merge_df = base_df_copy[['author', 'title']].drop_duplicates().merge(unique_text_words.groupby(['author', 'title']).size().reset_index(name='cross_text_unique_word_count'), on=['author', 'title'], how='left')
merge_df['cross_text_unique_word_count'] = merge_df['cross_text_unique_word_count'].fillna(0).astype(int)

text_df = text_df.merge(merge_df, on=['author', 'title'], how='left')

In [ ]:
# export word_df as parquet to bin folder
# word_df.to_parquet('../bin/scraped_latin_texts_word_data.parquet', index=False, compression="gzip", engine="pyarrow")

In [ ]:
# export author_df as parquet to bin folder
# author_df.to_parquet('../bin/scraped_latin_texts_author_data.parquet', index=False, compression="gzip", engine="pyarrow")

In [ ]:
# export text_df as parquet to bin folder
# text_df.to_parquet('../bin/scraped_latin_texts_text_data.parquet', index=False, compression="gzip", engine="pyarrow")